# Leak localization

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from natsort import natsorted
import networkx as nx
import epynet 

In [5]:
# --------------------------
# Importing custom libraries
# source: https://github.com/GardarGardarsson/GSP_for_Leak_Detection/tree/864a99041af6331375946a8fad3a71c633108eb8
# --------------------------

# Import a custom tool for converting EPANET .inp files to networkx graphs
from ep_utils.epanet_loader import get_nx_graph

# Function for visualisationa
from ep_utils.visualisation import visualise

# EPANET simulator, used to generate nodal pressures from the nominal model
from ep_utils.epanet_simulator import epanetSimulator

# SCADA timeseries dataloader
from ep_utils.data_loader import battledimLoader, dataCleaner, dataGenerator

In [6]:
path_to_wdn = './data/l-town-data/L-TOWN_Real.inp'

In [7]:
# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

In [8]:
G, _ = get_nx_graph(wdn, weight_mode='real_pipe_length', get_head=False)

In [9]:
neighbours_by_pipe = {}

for node in G:
    for neighbour, connecting_edge in G[node].items():
        if connecting_edge['name'] == 'SELF':
            continue
        else:
            neighbours_by_pipe[connecting_edge['name']] = [node, neighbour]
            
pipe_by_neighbours = { str(neighbour_list) : pipe for pipe , neighbour_list in neighbours_by_pipe.items()}

In [10]:
def dist_to_true(predict, true):
    distance = min(nx.shortest_path_length(G, source=predict, target=neighbours_by_pipe[true][0], weight='weight'),
                   nx.shortest_path_length(G, source=predict, target=neighbours_by_pipe[true][1], weight='weight'))
    return distance

In [11]:
folder = './reconstruct_error/'
files = natsorted(os.listdir(folder))
df_dict = {}
df_list = []
for file in files:
    if (file.split('.'))[-1] != 'csv':
        continue
    name, _ = os.path.splitext(file)
    filePath = folder + file
    df = pd.read_csv(filePath,index_col=0)
    df_dict[name] = df
    # 合并传感器
    df_row_sums = pd.DataFrame(df.sum(axis=1))
    df_row_sums.columns = [(file.split('.'))[0]]
    df_list.append(df_row_sums)

new_df = pd.concat(df_list,axis=1)
sensors_error_df = pd.concat(list(df_dict.values()),axis=1)

In [12]:
detect_dict = {
'zone1': [['2019-03-17 06:00:00', 'p280'], ['2019-06-05 15:15:00', 'p277']],
'zone2': [['2019-04-28 01:30:00', 'p331'], ['2019-11-06 19:40:00', 'p426']],
'zone3': [['2019-04-06 01:00:00', 'p710'], ['2019-08-31 10:50:00', 'p721']],
'zone4': [['2019-08-31 08:25:00', 'p800'], ['2019-12-07 11:15:00', 'p879']],
'zone5': [['2019-01-25 03:35:00', 'p827'],
['2019-06-14 00:25:00', 'p193'],
['2019-11-06 21:10:00', 'p762']],
'zone6': [['2019-07-10 15:05:00', 'p680']],
'zone7': [['2019-03-28 02:00:00', 'p653'], ['2019-06-13 17:45:00', 'p142'], ['2019-12-07 11:45:00','p123']],
'zone8': [['2019-01-16 13:20:00', 'p523'],
['2019-04-06 09:35:00', 'p514'],
['2019-08-31 10:15:00', 'p586']],
'zone9': [['2019-11-30 23:00:00', 'p455']],
'zone10': []
}

In [13]:
sensors_list = {

'zone1' : ['n1','n4','n31'],

'zone2' : ['n410','n429'],

'zone3' : ['n342','n636','n644'],

'zone4' : ['n296','n679','n722','n740'],

'zone5' : ['n288','n726','n752','n769'],

'zone6' : ['n215','n229'],

'zone7' : ['n163','n188','n613'],

'zone8' : ['n332','n495','n506','n549'],

'zone9' : ['n105','n114','n469', 'n516'],

'zone10' : ['n54','n415','n458', 'n519']
    
}

In [14]:
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0
    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity


def localize(df, target):
    max_similarity = -1
    min_similarity = 1
    max_key = None
    min_key = None

    for index, row in df.iterrows():
        similarity = cosine_similarity(row.values, target)
        if similarity > max_similarity:
            max_similarity = similarity
            max_key = index

        if similarity < min_similarity:
            min_similarity = similarity
            min_key = index

    return int(max_key[1:])


import heapq

def localize_top_k(df, target, k):
    similarity_list = []

    for index, row in df.iterrows():
        similarity = cosine_similarity(row.values, target)
        similarity_list.append((similarity, index))

    similarity_list.sort(key=lambda x: -x[0])

    top_k_similarities = [pair[0] for pair in similarity_list]
    top_k_indices = [pair[1] for pair in similarity_list]

    return top_k_similarities[:k], top_k_indices[:k]


In [15]:
import yaml
file_path = 'correlate_sensors.yaml'

# read YAML file
with open(file_path, 'r') as file:
    correlate_sensors = yaml.load(file, Loader=yaml.FullLoader)

In [16]:
# use correlate sensor info for localization
def localize_ver2(df, target, correlate_sensors):
    max_similarity = -1
    min_similarity = 1
    max_key = None
    min_key = None

    for index, row in df.iterrows():
        cur_corr_sensors = correlate_sensors[index]

        similarity = cosine_similarity(row[cur_corr_sensors].values, target[cur_corr_sensors])

        if similarity > max_similarity:
            max_similarity = similarity
            max_key = index

        if similarity < min_similarity:
            min_similarity = similarity
            min_key = index

    return int(max_key[1:])

In [17]:
simulate_folder = './impact_factor/factor_data/'
simulate_files = natsorted(os.listdir(simulate_folder))
leak_num = 19

In [18]:
true_cnt = 0
avg_dist = 0
use_window_target = True

categories_abrupt = ['p280', 'p331', 'p426', 'p710', 'p827', 'p680', 'p142', 'p523', 'p514']
categories_incipient = ['p277', 'p721', 'p800', 'p879', 'p193', 'p762', 'p653', 'p586', 'p455', 'p123']
dist_abrupt = []
dist_incip = []
cnt_ab = 0
cnt_in = 0

for file in simulate_files:
    name, _ = os.path.splitext(file)
    print(name)
    filePath = simulate_folder + file
    df = pd.read_csv(filePath,index_col=0)
    for detect in detect_dict[name]:
        t, pipe = detect
        target_vector = sensors_error_df.loc[t]
        
        if use_window_target:
            start_time = str(pd.Timestamp(t) - pd.Timedelta(days=0))
            end_time = str(pd.Timestamp(t) + pd.Timedelta(hours=1))

#             window_data = df_dict[name].loc[t:end_time]
            window_data = sensors_error_df.loc[start_time:end_time]

            # 计算时间窗口内的均值
            window_mean = window_data.mean()
#             target_vector = list(window_mean)
            target_vector = window_mean
            
#         predict = localize(df[sensors_list[name]], target_vector)
#         predict = localize(df[sensors_list[name]], target_vector[sensors_list[name]])
        predict = localize_ver2(df, target_vector, correlate_sensors)
        distance = dist_to_true(predict, pipe)
        avg_dist += distance
        
        cs_max = 0
        nei_max = -1
        for nei in list(G.neighbors(predict)):
            if f'n{nei}' not in df.index:
                continue
            cs = cosine_similarity((df.loc[f'n{nei}'])[correlate_sensors[f'n{nei}']].values, target_vector[correlate_sensors[f'n{nei}']])
            if cs > cs_max:
                cs_max = cs
                nei_max = nei
        predict_pipe = pipe_by_neighbours[str(sorted([predict, nei_max], reverse=True))]
        print(f'Leak: {pipe}, Predict: {predict}, Distance: {distance:.1f}, {distance<=300}')
        if distance<=300:
            true_cnt += 1
            
            
        if pipe in categories_abrupt:
            dist_abrupt.append(distance)
            if distance<=300:
                cnt_ab += 1
        else:
            dist_incip.append(distance)
            if distance<=300:
                cnt_in += 1
            
avg_dist /= leak_num
print(f'Localized: {true_cnt}')
print(f'Avg distance: {avg_dist:.2f}')
avg_ab = sum(dist_abrupt)/len(dist_abrupt)
avg_in = sum(dist_incip)/len(dist_incip)
print(f'Localized: {true_cnt}, Abrupt: {cnt_ab}, Incipient: {cnt_in}')
print(f'Avg dist: {avg_dist:.2f}, Abrupt: {avg_ab:.2f}, Incipient: {avg_in:.2f}')

zone1
Leak: p280, Predict: 31, Distance: 50.2, True
Leak: p277, Predict: 31, Distance: 414.3, False
zone2
Leak: p331, Predict: 438, Distance: 104.2, True
Leak: p426, Predict: 82, Distance: 226.0, True
zone3
Leak: p710, Predict: 643, Distance: 172.2, True
Leak: p721, Predict: 644, Distance: 299.6, True
zone4
Leak: p800, Predict: 758, Distance: 174.7, True
Leak: p879, Predict: 765, Distance: 0.0, True
zone5
Leak: p827, Predict: 731, Distance: 0.0, True
Leak: p193, Predict: 774, Distance: 122.8, True
Leak: p762, Predict: 774, Distance: 946.5, False
zone6
Leak: p680, Predict: 209, Distance: 74.5, True
zone7
Leak: p653, Predict: 613, Distance: 197.0, True
Leak: p142, Predict: 608, Distance: 292.0, True
Leak: p123, Predict: 182, Distance: 296.3, True
zone8
Leak: p523, Predict: 506, Distance: 66.1, True
Leak: p514, Predict: 517, Distance: 0.0, True
Leak: p586, Predict: 549, Distance: 170.0, True
zone9
Leak: p455, Predict: 484, Distance: 244.5, True
zone10
Localized: 17
Avg distance: 202.68
Lo